In [1]:
from CurriculumVitae import *
from IPython.display import display, HTML, Markdown
from datetime import date

In [2]:
my_resume = Resume(
    person=Person(name="Zach Allen", pronouns="He/Him", 
                    contact_info=ContactInfo(
                       email="fractalmachinist@gmail.com", 
                       phone="+1 (509)438-8146", 
                       link="https://linkedin.com/in/zachallen-fractalmachinist/",
                       link2="https://fractalmachini.st")),
    headline="Computer Scientist with 3 years experience in leadership, Data Science, and problem-solving. \
Reliably fast learner across all topics. Independently implemented and deployed terabyte-scale AWS genomics pipeline in 4 months part-time. \
Developed novel ML architectures for unbalanced datasets (>1000:1), raising rare-case detection from 15% to 90% with no loss in accuracy. \
Leverages communication and Project Management skills to integrate SME, business, and stakeholder needs.")

In [3]:
my_resume.education = [Occupation(
    title="Bachelor's in Computer Science", location="Utah, USA (Online)",
    timespan=Between(start=date(2019, 2, 1), end=date(2022, 8, 31)),
    subtitle="Western Governors University", 
    headline="Thrived in a self-directed remote envirionment",
    sub_tasks=[
        Effort(title="Diamond Price Prediction Model",
               headline="Produced and documented a diamond price prediction model.",
               website="https://github.com/FractalMachinist/WGU-C968",
               achievements=[
                   Achievement(
                       headline="Developed simple Data Pipeline in python/pandas.",
                       skills=my_resume.Skills("Python", "Data Engineering", "Software Engineering")),
                   Achievement(
                       headline="Demonstrated integration of business needs and SDLC processes.",
                       skills=my_resume.Skills("Python", "Project Management", "Technical Communication", "SDLC", "Documentation", "Value Delivery", "Requirement Management"))]
        ), 
        Effort(title="Advanced Java Concepts",
               headline="Developed appointment scheduling and customer database tool in Java.",
               achievements=[Achievement(
                   headline="Demonstrated tight integration with JavaFX, JDBC, and MySQL.",
                   skills=my_resume.Skills("Java", "SQL", "OOP", "JavaFX", "JDBC", "MySQL"))])],
    
    achievements=[
        
        my_resume.Certification(name="CompTIA Project+",
                      headline="Demonstrated understanding of Project Management roles, processes, and documentation.",
                      issuer="Pearson VUE", issue_date=date(2019, 7, 24),
                      website="https://wsr.pearsonvue.com/testtaker/authenticate/AuthenticateScoreReport.htm",
                      confirmation_info={"Registration":"358639011", "Validation":"155946649"},
                      skills=my_resume.Skills("Project Management", "Technical Communication", "Requirement Management", "SDLC")),
        
        my_resume.Certification(name="IT Information Library Foundations Certification (ITIL)", 
                      headline="Demonstrated understanding of designing, deploying, maintaining, and retiring IT resources.",
                      issuer="AXELOS", issue_date=date(2020, 8, 1),
                      skills=my_resume.Skills("Service Management", "Requirement Management", "Value Delivery", "SDLC")),
        
        my_resume.Certification(name="Site Development Associate",
                      # exports=["html"],
                      headline="Demonstrated ability to design and build websites.",
                      issuer="CIW", issue_date=date(2019, 2, 1),
                      skills=my_resume.Skills("Web Development", "Network Administration")),
        
        Achievement(headline="Excellence Award for Communication Applications",
                    skills=my_resume.Skills("Technical Communication")),
        
    ])]

In [4]:
my_resume.employment = [
    Occupation(title="Sofware Engineer", location="St. Louis, Missouri, USA", website="https://PlutonBio.com",
               timespan=Between(start=date(2021, 3, 1), end=date(2021, 8, 31)),
               subtitle="Pluton Biosciences", supervisor=Person(name="Dr. Boahemaa Adu-Oppong", pronouns="She/Her",
                                                                contact_info=ContactInfo(email="BAdu-Oppong@plutonbio.com")),
               
               headline="Data Engineering supporting bioinformatics research",               
               achievements=[
                   Achievement(headline="Independently designed, implemented, and deployed Terabyte-scale AWS genomics pipeline in 4 months, from no prior genomics or AWS experience.",
                               skills=my_resume.Skills("Software Engineering", "Python", "AWS", "Docker/Kubernetes", "Data Engineering", "Rapid Learning", "Linux/Bash", "Project Management", "Self-Direction", "Optimization", "Test Automation & Validation", "Biotechnology", "Bioinformatics")),

                   Achievement(headline="Interfaced with SMEs to gather, interpret, and execute on pipeline requirements. Interfaced with Bioinformatics pipeline design expert for high-level guidance.",
                               skills=my_resume.Skills("Requirement Management", "Technical Communication", "Project Management")),

                   Achievement(headline="Designed, implemented, developed training for, and documented an in-house Python API enabling (non-CS) biologists to deploy multi-stage genomics queries without relying on the CS team.",
                               skills=my_resume.Skills("Software Engineering", "Python", "Technical Communication", "Documentation", "Value Delivery", "Service Management", "Requirement Management")),
                   
                   Achievement(headline="Led the CS team in adopting CI/CD tools like Git, AWS CodeCommit, and Docker / AWS Elastic Container Registry.",
                               skills=my_resume.Skills("Leadership", "Service Management", "Value Delivery", "CI/CD", "SDLC", "Git", "Docker/Kubernetes", "AWS", "Test Automation & Validation"))]),
    
    Occupation(title="Embedded Systems Engineer", location="Allentown, Pennsylvania, USA", website="https://AppliedSeparations.com",
               timespan=Between(start=date(2019, 1, 1), end=date(2020, 1, 25)),
               subtitle="Applied Separations", supervisor=Person(name="Aaron Allen", pronouns="He/Him", 
                                                                contact_info=ContactInfo(email="mnrnln@gmail.com")),
               
               headline="Developed custom C++ / Arduino pump control software",
               achievements=[
                   Achievement(headline="Designed and developed pump control software for chromatography and analytical chemistry systems.",
                               skills=my_resume.Skills("C++", "Software Engineering", "SDLC")),
                   
                   Achievement(headline="Wrote a simple heuristic scheduler & virtual threading to manage real-time (60Hz+) pump control, touch screen input, and data/control communication, all on a single Arduino Mega.",
                               skills=my_resume.Skills("Algorithm Design & Selection", "Optimization", "Software Engineering")),
                   
                   Achievement(headline="Designed and validated mass delivery tracking, prediction, and smoothing algorithms, to handle nonlinear feedback delay.",
                               skills=my_resume.Skills("Algorithm Design & Selection", "Mathematics", "Software Engineering", "Test Automation & Validation", "Data Engineering")),
                   
                   Achievement(headline="Interfaced with separate development team in charge of chromatography control software. Collected and implemented requirements from instrumentation engineer.",
                               skills=my_resume.Skills("Technical Communication", "Teamwork", "Requirement Management", "Project Management", "SDLC", "Documentation"))]),
              
    Occupation(title="Service Writer", location="Richland, Washington, USA", website="https://AlphaComputerCenter.com",
               # exports=["html"],
               timespan=Between(start=date(2017, 12, 1), end=date(2018, 11, 30)),
               subtitle="Alpha Computer Center", supervisor=Person(name="Frank Ward Jr.", pronouns="He/Him",
                                                                   contact_info=ContactInfo(email="frankjr@alphacomputercenter.com",
                                                                   phone="+1 (509)946-4230")),
               
               headline="Customer service, sales, and technician support",
               achievements=[
                   Achievement(headline="Ensured customers were able to accurately understand and communicate with repair technicians, improving customer service and reducing diagnostic time.",
                               skills=my_resume.Skills("Technical Communication")),
                   
                   Achievement(headline="Leveraged extensive Linux experience to rapidly identify and repair issues that couldn't be fixed by Mac diagnostic tools.",
                               skills=my_resume.Skills("Linux/Bash")),
                   
                   Achievement(headline="Reduced call frequency with an informative website. See it on <a href='https://web.archive.org/web/20180113192132/http://www.alphacomputercenter.com/wordpress1/'>web archive</a>.",
                               skills=my_resume.Skills("Web Development", "Graphic Design"))]),
              
    Occupation(title="Machine Learning Researcher (Intern)", location="Richland, Washington, USA", website="https://pnnl.gov",
               timespan=Between(start=date(2017, 1, 1), end=date(2017, 5, 31)),
               subtitle="Pacific Northwest National Laboratory", supervisor=Person(name="Dr. Enoch Yeung", pronouns="He/Him",
                                                                                   contact_info=ContactInfo(email="eyeung@ucsb.edu")),
               
               headline="ML Research and Data Engineering intern",
               achievements=[
                   Achievement(headline="Designed and tested novel Neural Network algorithms, architectures, and error formulations for NLP, image classification, and time-series data classification.",
                               skills=my_resume.Skills("Python", "TensorFlow", "Machine Learning", "Neural Networks", "Algorithm Design & Selection", "Mathematics", "NLP", "Data Engineering", "Rapid Learning")),
                   
                   Achievement(headline="Demonstrated increased test accuracy (15% ⇾ 90% detection with higher Bayesian Confidence) on unbalanced (>1000:1) datasets, without duplication, augmentation, or batch filtering.",
                               skills=my_resume.Skills("Machine Learning", "Mathematics", "Statistics", "Test Automation & Validation", "Experimental Design", "Algorithm Design & Selection")),
                   
                   Achievement(headline="Worked independently, balancing multiple projects and deliverables with minimal mentor supervision, often meeting every two weeks.",
                               skills=my_resume.Skills("Project Management", "Technical Communication"))])
]

In [5]:
my_resume.projects = [
    Effort(title="Neural Cellular Segmentation",
           headline="Exploring neural cellular automata and attention (NCA+A) for medical image segmentation",
           website="https://github.com/FractalMachinist/NeuralCellularAutomataAttn",
           sub_tasks=[
               Achievement(headline="Developed, tested, and iterated NCA+A models, balancing system resources and model size.",
                           skills=my_resume.Skills("Python", "TensorFlow", "Machine Learning", "Neural Networks", "Experimental Design", "Mathematics", "Optimization")),
               Achievement(headline="Created multiple tf.Data pipelines with preprocessing and data augmentation steps.",
                           skills=my_resume.Skills("Python", "Data Engineering", "Software Engineering", "TensorFlow")),
           ]),
    Effort(title="Interplan",
           # exports=["html"],
           headline="Task dependency management from a Graph Database",
           website="https://fractalmachini.st/demos/interplan",
           achievements=[
               Achievement(headline="Developed a Neo4J+React dependency resolution and task status tracking web app.",
                           skills=my_resume.Skills("React", "Test Automation & Validation", "Project Management", "Web Development")),
               Achievement(headline="Packaged React app and Neo4J database in Docker & Kubernetes for easy migration.",
                           skills=my_resume.Skills("Docker/Kubernetes", "Linux/Bash"))
           ]),
    
    Effort(title="MarkNotes",
           # exports=["html"],
           headline="Intuitive journaling tool designed to encourage long-term review and introspection",
           website="https://fractalmachini.st/demos/marknotes",
           achievements=[
               Achievement(headline="Implemented MongoDB and Node API for destructuring, storing, and querying Markdown entries as semi-structured data.",
                           skills=my_resume.Skills("MongoDB", "Data Engineering")),
               Achievement(headline="Packaged React app and MongoDB in Docker & Kubernetes.",
                           skills=my_resume.Skills("Docker/Kubernetes", "Linux/Bash"))
           ]),
    
    Effort(title="NetTimeLog",
           # exports=["html"],
           headline="Minimalist, accurate time tracking",
           website="https://fractalmachini.st/demos/nettimelog",
           achievements=[
               Achievement(headline="Created time-tracking web app which records what you just completed, so you never estimate what you will do or how long it will take.",
                           skills=my_resume.Skills("Python", "Flask", "Web Development"))])
]

In [31]:
job_listings = [
    JobListing("Inari_SWE")
]

In [32]:
display(Markdown("\n".join([f"- [ ] {listing.export(my_resume)}" for listing in job_listings])))

- [ ] http://fractalmachini.st/pdf_sources/Inari_SWE.html

In [29]:
my_resume.write_html_to_file(
    filepath="docs/index.html",
    jinja2_render_args={
        "stylesheet":"chalkboard"
    },
    popularity_dict={
        Skill: {
            "python": 5,
            "tensorflow":4,
            "neural networks":3.9,
            "data engineering":3.8,
            "machine learning":3.7,
            "docker/kubernetes":3,
            "software engineering":2
        }
    }
)

In [30]:
my_resume.write_html_to_file(
    filepath="docs/resumes/index.html",
    popularity_dict={
        Skill: {
            "python": 5,
            "tensorflow":4,
            "neural networks":3.9,
            "data engineering":3.8,
            "machine learning":3.7,
            "docker/kubernetes":3,
            "software engineering":2
        }
    }
)